# Variational Quantum Classifier (VQC) Hardware

The notebook runs a feature map on real quantum hardware

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [ ]:
# imports
from matplotlib import pyplot as plt
import seaborn as sns

import os
import time

# import data class
from utilities.dataset_utils import DiabetesData

# qiskit imports
# vqc for feature map call
from qiskit_machine_learning.algorithms import VQC

# hardware execution
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Sampler

# ciruit transpiler
from qiskit import transpile

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [7]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [8]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'models', 'qml-simulator', 'vqc_best_params.model')

vqc = VQC.load(path)

In [9]:
service = QiskitRuntimeService()

In [11]:
filtered_backends = service.backends(
    simulator=False,
    operational=True,
    min_num_qubits=8
)

print("Available Backends: ", [backend.name for backend in filtered_backends])

Available Backends:  ['ibm_brisbane', 'ibm_kyiv', 'ibm_sherbrooke']


In [12]:
backend_name = 'ibm_brisbane'

backend = service.backend(backend_name)

print(f'Backend: {backend_name}\nNum Qubits: {backend.num_qubits}')

Backend: ibm_brisbane
Num Qubits: 127


In [13]:
# feature map
feature_map = vqc.feature_map

In [ ]:
# get data
X_train, X_test, y_train, y_test = dataset.preprocess_data_ranged()

In [14]:
feature_map.parameters

ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4]), ParameterVectorElement(x[5]), ParameterVectorElement(x[6]), ParameterVectorElement(x[7])])

In [15]:
feature_map.draw()

┌───┐┌─────────────┐┌───┐┌─────────────┐
0: ┤ H ├┤ P(2.0*x[0]) ├┤ H ├┤ P(2.0*x[0]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
1: ┤ H ├┤ P(2.0*x[1]) ├┤ H ├┤ P(2.0*x[1]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
2: ┤ H ├┤ P(2.0*x[2]) ├┤ H ├┤ P(2.0*x[2]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
3: ┤ H ├┤ P(2.0*x[3]) ├┤ H ├┤ P(2.0*x[3]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
4: ┤ H ├┤ P(2.0*x[4]) ├┤ H ├┤ P(2.0*x[4]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
5: ┤ H ├┤ P(2.0*x[5]) ├┤ H ├┤ P(2.0*x[5]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
6: ┤ H ├┤ P(2.0*x[6]) ├┤ H ├┤ P(2.0*x[6]) ├
   ├───┤├─────────────┤├───┤├─────────────┤
7: ┤ H ├┤ P(2.0*x[7]) ├┤ H ├┤ P(2.0*x[7]) ├
   └───┘└─────────────┘└───┘└─────────────┘

In [ ]:
# transpiled feature map
feature_map_transpiled = transpile(feature_map, backend=backend, optimization_level=3)

In [ ]:
# sampler
sampler = Sampler(mode=backend)

job = sampler.run([feature_map_transpiled], shots=1024)